In [1]:
# Import libraries
import json
from datetime import datetime
import pandas as pd
# Library to correct accents, remove emoticons and set sentiment according emoticons
import funciones

In [2]:
# Load posts data
with open('posts/your_posts_1.json') as f:
  data = json.load(f)

In [3]:
# Data format
print(type(data))
# View content format
#print(data)

<class 'list'>


In [4]:
# Explore few posts structure
print(data[7976])
list(data[7976])

{'timestamp': 1548085477, 'attachments': [{'data': [{'media': {'uri': 'photos_and_videos/Lunarojaene20_vUt3ZKqkjQ/49814837_10157026090202070_2927677389464928256_o_10157026090197070.jpg', 'creation_timestamp': 1548085471, 'media_metadata': {'photo_metadata': {'upload_ip': '181.131.165.119'}}, 'title': 'Luna roja - ene 20'}}]}], 'title': 'Isabel Yepes added a new photo.'}


['timestamp', 'attachments', 'title']

In [5]:
# Strip to the end
def right_char(text,char_stop,char_start):
    if text.find(char_stop,text.find(char_start)) == -1:
        return ""
    else:
        return text[text.find(char_stop,text.find(char_start)):]

In [6]:
# Remove mentions
def filter_mention(content):
    temp = content
    while temp.find("@") > -1:
        if temp.find("]",temp.find("@")) == -1:
            temp = ""
        else:
            temp = temp[:temp.find("@")] + temp[temp.find("]",temp.find("@"))+1:]
    return temp

# Remove urls
def filter_url(content):
    temp = content
    while temp.find("http") > -1:
        temp = temp[:temp.find("http")] + right_char(temp," ","http")
    return temp

In [7]:
# Fill dataframe with posts data with different dict formats and add sentiment using emoticons

def fill_dataframe(start,stop,data_array):
    posts_array = []
    posts_raw_array = []
    fechas = []
    sentiment = []
    for i in range(start, stop):
        whole_dict = data_array[i]
        post_datetime = datetime.fromtimestamp(whole_dict['timestamp'])
        if 'data' in whole_dict:
            data_dict = whole_dict['data']
            if data_dict:
                post = data_dict[0]
                if list(post) == ['post']:
                    # Save post before changing chars format
                    posts_raw_array.append(post['post'])
                    # Get post polarity
                    sent = funciones.emoticons_sentiment(post['post'])
                    sentiment.append(str(sent))
                    # Clean post for Spanish and Analysis
                    post = funciones.accent_replace(post['post'])
                    post = funciones.emoticons_replace(post)
                    post = filter_mention(filter_url(post))
                    posts_array.append(post)
                    fechas.append(str(post_datetime))
                else:
                    post = "no_post"
            else:
                post = "no_post"
        elif 'attachments' in whole_dict:
            data_dict = whole_dict['attachments']
            data_media = data_dict[0]
            media_dict = data_media['data']
            media_detail = media_dict[0]
            media_data = media_detail['media']
            if 'description' in media_data:
                # Save post before changing chars format
                posts_raw_array.append(media_data['description'])
                # Get post polarity
                sent = funciones.emoticons_sentiment(media_data['description'])
                sentiment.append(str(sent))
                # Clean post for Spanish and Analysis
                post = funciones.accent_replace(media_data['description'])
                post = funciones.emoticons_replace(post)
                post = filter_mention(filter_url(post))
                posts_array.append(post)
                fechas.append(str(post_datetime))
            elif 'title' in media_data:
                # Save post before changing chars format
                posts_raw_array.append(media_data['title'])
                # Get post polarity
                sent = funciones.emoticons_sentiment(media_data['title'])
                sentiment.append(str(sent))
                # Clean post for Spanish and Analysis
                post = funciones.accent_replace(media_data['title'])
                post = funciones.emoticons_replace(post)
                post = filter_mention(filter_url(post))
                posts_array.append(post)
                fechas.append(str(post_datetime))
            else:
                post = "no_post"
        else:
            post = "no_post"
    df_content = {'dates' : fechas[:], 'content': posts_array[:], 'sentiment': sentiment[:]}
    df = pd.DataFrame(df_content)
    # Create raw dataframe for saving
    df_raw_content = {'dates' : fechas[:], 'content': posts_raw_array[:], 'sentiment': sentiment[:]}
    df_raw = pd.DataFrame(df_raw_content)
    #print(df_raw)
    file_range = '-' + str(start) + '-' + str(stop)
    # Save raw data with empty posts stripped
    df_raw.to_csv(r'results/post_filtered' + file_range + '.csv', index = False)
    # Show dates included in the range
    print('Datos Desde: ' + df.loc[0,'dates'] + ' Hasta: ' + df.iloc[-1,0])
    return df

In [8]:
# Example of data processed
df = fill_dataframe(0,10000,data)
df.head()

Datos Desde: 2020-09-01 20:57:46 Hasta: 2018-10-21 11:45:42


,dates,content,sentiment
0,2020-09-01 20:57:46,"Lindo, con nuestros impuestos.",none
1,2020-08-31 02:15:27,Hagamos famosa a Diana Trujillo,none
2,2020-08-30 08:02:41,18 años ya. Cuando esté viejita y ya nadie se ...,none
3,2020-08-30 07:17:53,Desde la casa también meten micos,none
4,2020-08-29 20:44:27,El triunfo de la energía estática sobre la fue...,1


In [9]:
# Verify frequencies of each category
df['sentiment'].value_counts()

none    4013
-1       552
1        405
0         49
Name: sentiment, dtype: int64

In [10]:
# Create a copy to not modify original data
train_df = df[['content','sentiment']].copy()
# Remove columns with no sentiment value or too few samples 
indexNames = train_df[(train_df['sentiment'] == 'none') | (train_df['sentiment'] == '0')].index
train_df.drop(indexNames , inplace=True)
train_df.head()

,content,sentiment
4,El triunfo de la energía estática sobre la fue...,1
8,""" ð¨ð´ El concurso, en el que participaro...",1
24,Les comparto este envivo de IG con Información...,-1
30,Alguien que Tuitea en vivo y en directo,1
32,"Debí escibir el nombre de la señora, ya no la ...",-1


In [11]:
# Import NLP library
import spacy
from spacy import displacy

In [12]:
# Load Spacy corpus
nlp = spacy.load('es_core_news_sm')

In [13]:
# Stop Words from es_core_news_sm
from spacy.lang.es.stop_words import STOP_WORDS
stopwords_spacy = list(STOP_WORDS)
len(stopwords_spacy)

551

In [14]:
#Importar librerías de aprendizaje
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [15]:
#Verificar si hay datos nulos
train_df.isnull().sum()

content      0
sentiment    0
dtype: int64

In [16]:
#Constante de signos de puntuación
import string
puntua = string.punctuation + '¡¿'
puntua

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¡¿'

In [17]:
#Función para limpieza de datos
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.strip()
        else:
            temp = token
        tokens.append(temp)
    
    clean_tokens = []
    for token in tokens:
        if token not in stopwords_spacy and token not in puntua:
            clean_tokens.append(token)
    
    return clean_tokens

In [18]:
# Test cleaning
print(train_df['content'][4])
text_data_cleaning(train_df['content'][4])

El triunfo de la energía estática sobre la fuerza de gravedad     


['El', 'triunfar', 'energía', 'estático', 'sobrar', 'forzar', 'gravedad']

In [19]:
# Importar vectorization library
from sklearn.svm import LinearSVC

In [20]:
# Define tokenize function and create classifier
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [21]:
# Create datos vectors
X = train_df['content']
y = train_df['sentiment']

In [22]:
# Create training vector as a portion of data leave the rest for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((765,), (192,), (765,), (192,))

In [23]:
X_train.head()

1909                                                    "
2745                            Ay mi América Latina     
1873    She asked for memes with her car photo and she...
3054                                                     
2737                                                     
Name: content, dtype: object

In [24]:
# Create pipeline
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [25]:
# Define format explicitly to avoid being taken as unknown
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [26]:
# Train classifier
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [27]:
# Create predicted vector from classifier
y_pred = clf.predict(X_test)

In [28]:
# Verify precision obtained
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.69      0.93      0.80       106
           1       0.86      0.49      0.62        86

   micro avg       0.73      0.73      0.73       192
   macro avg       0.77      0.71      0.71       192
weighted avg       0.77      0.73      0.72       192



In [29]:
# Create confusion matrix
confusion_matrix(y_test, y_pred)

array([[99,  7],
       [44, 42]])

In [30]:
# Predict some post sentiments
print(df.loc[16,'content'])
clf.predict([df.loc[16,'content']])

Nunca habúa visto cómo lucía esta mariposa en su estado de oruga.


array([1])

In [31]:
print(df.loc[160,'content'])
clf.predict([df.loc[160,'content']])

Por esto me toca comprar agua para cocinar en Bogotá, en cualquier momento sin aviso sale amarilla por la llave.  De hecho la cámara la muestra más clara de lo que es.


array([-1])

In [32]:
print(df.loc[0,'content'])
clf.predict([df.loc[0,'content']])

Lindo, con nuestros impuestos.


array([-1])

In [33]:
print(df.loc[9,'content'])
clf.predict([df.loc[9,'content']])

Ciencia para todas las personas, no sólo la élite científica


array([1])